<a href="https://colab.research.google.com/github/kasikotnani23/Kasi-k/blob/main/Working_with_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Working with DF (DataFrame)**

**`Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark`**

**`Author: Amin Karami (PhD, FHEA)`**

---

**DataFrame (DF)**: Schema (named columns) + declarative language. A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. It is very efficient for strucutred data.

source: https://spark.apache.org/docs/latest/sql-programming-guide.html

source: https://spark.apache.org/docs/latest/api/python/reference/

In [1]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=6e6c34371e8c14500f125a74f8e5cf14608eee6dc8ec57588422bda1aa4fb084
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
# Linking with Spark (https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html)
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

spark

# **Part 1: Create DF and Basic Operations**

In [4]:
# Create/Load DF: (Spark automatically scans through the files and infers the schema of the dataset)
# data source: https://www.kaggle.com/thec03u5/fifa-18-demo-player-dataset

df1 = spark.read.format("csv").load("CompleteDataset.csv", inferSchema=True, header=True)

In [ ]:
# Show data:
df1.show()

In [ ]:
# How many partitions in DF?
df1.rdd.getNumPartitions()

In [ ]:
# Increase/Desrease the partitions in DF
df2 = df1.repartition(4)
df2.rdd.getNumPartitions()

In [ ]:
# Show DF
df2.show()

In [ ]:
# Rename Columns and Amend NULLs:
df2 = df2.withColumnRenamed("_c0", "ID") \
    .withColumnRenamed("Ball control", "Ball_Control")\
    .withColumnRenamed("Sliding tackle", "Sliding_Tackle")

df2.na.fill({"RAM": 10, "RB": 1}).show()

In [ ]:
# Transformation (SELECT):
df2.select("Name","Overall").distinct().show()

In [ ]:
# Transformation (FILTER):
df2.filter(df2["Overall"] > 70).show()

In [ ]:
# Transformation (FILTER):
df2.select("Overall", "Name", "Age").where(df2["Overall"]>70).show()

In [ ]:
# Transformation (FILTER):
df2.where(df2["Overall"]>70).groupBy("Age").count().sort("Age").show()

In [ ]:
# Visualize the results:
df2_result = df2.where(df2["Overall"]>70).groupBy("Age").count().sort("Age")

pandas_df = df2_result.toPandas()
pandas_df.plot(x = "Age", y = "count", kind = "bar")


In [ ]:
pandas_df.sort_values(by="count", ascending=False).plot(x = "Age", y = "count", kind = "bar")

# **Part 2: Advanced DF Operations: Spark SQL and UDF**

In [16]:
# Spark SQL (Register the DF using a local temporary view):

df2.createOrReplaceTempView("df_football")

In [ ]:
# SQL Query:

sql_query = """ SELECT Age, count(*) as Count
                FROM df_football
                WHERE Overall > 70
                GROUP BY Age
                ORDER BY Age """

result = spark.sql(sql_query)
result.show()

In [ ]:
# UDF (User Defined Functions):
def uppercase_converter(record):
  if record is not None:
    if len(record) > 10:
      return record.upper()
    else:
      return record.lower()

# register the DF
df2.createOrReplaceTempView("UDF_football")

# register the function
spark.udf.register("UPPER", uppercase_converter)

# use the UDF in SQL
sql_query = "SELECT Age, UPPER(Name) as Name, UPPER(Club) as Club FROM UDF_football"

result = spark.sql(sql_query)
result.show()
